In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras import models
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
import os
import copy

In [2]:
def cross_validate_and_select_best_params(model, X, y, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    scores = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': []
    }

    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # print(f"y_train shape = {y_train.shape}, y_test shape = {y_test.shape}")
        history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_test = np.argmax(y_test, axis=1) 
        
        scores['accuracy'].append(accuracy_score(y_test, y_pred))
        scores['precision'].append(precision_score(y_test, y_pred, average='macro'))
        scores['recall'].append(recall_score(y_test, y_pred, average='macro'))
        scores['f1'].append(f1_score(y_test, y_pred, average='macro'))
        scores['roc_auc'].append(roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average='macro'))  

    avg_scores = {key: np.mean(value) for key, value in scores.items()}
    return avg_scores

In [3]:
X_combined = np.load('large_noise_cifar10.npy')
y_combined = np.load('train_labels.npy')

In [4]:
def build_simple_cnn(input_shape, num_classes, learning_rate=0.0005, num_filter=16, filter_size=(3, 3)):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  
    model.add(layers.Conv2D(num_filter, filter_size, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
print(y_combined.shape)
print(X_combined.shape)

(60000,)
(60000, 32, 32, 3)


In [6]:
input_shape = (32, 32, 3)
num_classes = 10

num_filters = [8]  
learning_rates = [0.001, 0.0005, 0.0015, 0.01, 0.005]  
filter_sizes = [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)]  

highest_config = []
highest_accuracy = 0

for num_filter in num_filters:
    for learning_rate in learning_rates:  
        highest_filter_size = (0, 0)
        highest_filter_size_accuracy = 0
        best_filter_size = False
        
        for filter_size in filter_sizes:  
            current_config = [learning_rate, num_filter, filter_size]
            model = build_simple_cnn(input_shape, num_classes, learning_rate, num_filter, filter_size)

            X_combined_reshaped = X_combined.reshape(-1, 32, 32, 3)
            y_combined_categorical = to_categorical(y_combined, num_classes=10)
            X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0

            with tf.device('/GPU:0'):
                avg_scores = cross_validate_and_select_best_params(model, X_combined_reshaped, y_combined_categorical)

            print("current_config: ", current_config)
            print(avg_scores)

            if avg_scores['accuracy'] >= highest_filter_size_accuracy:
                highest_filter_size_accuracy = avg_scores['accuracy']
                highest_filter_size = current_config[2]
                best_filter_size = False
            elif highest_filter_size_accuracy > avg_scores['accuracy'] and filter_size > highest_filter_size:
                print("Last filter_size is best, break")
                best_filter_size = True

            if avg_scores['accuracy'] > highest_accuracy:
                highest_config = current_config
                highest_accuracy = avg_scores['accuracy']
                print("highest_config: ", highest_config)
                print("highest_accuracy: ", highest_accuracy)

            if best_filter_size:
                break

print("highest_config: ", highest_config)
print("highest_accuracy: ", highest_accuracy)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3486 - loss: 1.8416 - val_accuracy: 0.4202 - val_loss: 1.7480
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4864 - loss: 1.4436 - val_accuracy: 0.4873 - val_loss: 1.4606
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5312 - loss: 1.3290 - val_accuracy: 0.4582 - val_loss: 1.5496
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5595 - loss: 1.2477 - val_accuracy: 0.5092 - val_loss: 1.4169
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5821 - loss: 1.1879 - val_accuracy: 0.5178 - val_loss: 1.4106
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5796 - loss: 1.2054 - val_accuracy: 0.5912 - val_loss: 1.1681
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6003 - loss: 1.1481 - val_accuracy: 0.5809 - val_loss: 1.1934
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6188 

In [7]:
input_shape = (32, 32, 3)
num_classes = 10

num_filters=[8, 16, 32, 64, 128]
best_learning_rate=highest_config[0]
best_filter_sizes=highest_config[2]

highest_config_last=[]
highest_accuracy_last=0
for num_filter in num_filters:
    current_config = [best_learning_rate, num_filter, best_filter_sizes]
    model = build_simple_cnn(input_shape, num_classes, best_learning_rate, num_filter, best_filter_sizes)

    X_combined_reshaped = X_combined.reshape(-1, 32, 32, 3)
    y_combined_categorical = to_categorical(y_combined, num_classes=10)
    X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0
    with tf.device('/GPU:0'):
        avg_scores = cross_validate_and_select_best_params(model, X_combined_reshaped, y_combined_categorical)
    print("current_config: ", current_config)
    print(avg_scores)

    if avg_scores['accuracy'] > highest_accuracy_last:
        highest_config_last = current_config
        highest_accuracy_last = avg_scores['accuracy']
        print("highest_config_last: ", highest_config_last)
        print("highest_accuracy_last: ", highest_accuracy_last)
            
print("highest_config_last: ", highest_config_last)
print("highest_accuracy_last: ", highest_accuracy_last)

Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3659 - loss: 1.7935 - val_accuracy: 0.4109 - val_loss: 1.6809
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5211 - loss: 1.3604 - val_accuracy: 0.4950 - val_loss: 1.4037
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5686 - loss: 1.2168 - val_accuracy: 0.4905 - val_loss: 1.4700
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6023 - loss: 1.1305 - val_accuracy: 0.5387 - val_loss: 1.3102
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6298 - loss: 1.0477 - val_accuracy: 0.5232 - val_loss: 1.3728
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6320 - loss: 1.0610 - val_accuracy: 0.6445 - val_loss: 1.0118
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6620 - loss: 0.9811 - val_accuracy: 0.6203 - val_loss: 1.0732
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6761 